# 5.1 순서가 없는 범주형 특성 인코딩하기

In [1]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

In [2]:
feature = np.array([['Texas'],
                   ['California'],
                   ['Texas'],
                   ['Delaware'],
                   ['Texas']])

In [3]:
# 원-핫 인코더
one_hot = LabelBinarizer()

In [4]:
# 특성을 원-핫 인코딩 한다.
one_hot.fit_transform(feature)

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

In [5]:
# 특성 클래스 확인하기
one_hot.classes_

array(['California', 'Delaware', 'Texas'], dtype='<U10')

In [6]:
# 원-핫 인코딩 되돌리기
one_hot.inverse_transform(one_hot.transform(feature))

array(['Texas', 'California', 'Texas', 'Delaware', 'Texas'], dtype='<U10')

In [7]:
import pandas as pd

In [8]:
pd.get_dummies(feature[:, 0])

,California,Delaware,Texas
0,0,0,1
1,1,0,0
2,0,0,1
3,0,1,0
4,0,0,1


In [9]:
# 다중 클래스 특성 만들기
multiclass_feature = [('Texas', 'Florida'),
                      ('California', 'Alabama'),
                      ('Texas', 'Florida'),
                      ('Delware', 'Florida'),
                      ('Texas', 'Alabama')]

In [10]:
# 다중 클래스 원-핫 인코더
one_hot_multiclass = MultiLabelBinarizer()

In [11]:
# 다중 클래스 특성을 원-핫 인코더 하기
one_hot_multiclass.fit_transform(multiclass_feature)

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

In [12]:
# 클래스 확인
one_hot_multiclass.classes_

array(['Alabama', 'California', 'Delware', 'Florida', 'Texas'],
      dtype=object)

범주가 순서에 영향이 있는지 없는지 먼저 판단한다!\
예를 들면 Texas:1 는 Alabama:2 보다 작지 않다.\
따라서 특성을 먼저 잘 파악한다.

LabelBinerizer: 문자열 타깃 데이터를 원-핫 인코딩으로 변환할 때 사용\
LabelEncoder: 문자열 타깃 데이터를 정수 레이블로 변환할 때 사용

In [13]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [14]:
feature = np.array([['Texas', 1],
                    ['California', 1],
                    ['Texas', 3],
                    ['Delaware', 1],
                    ['Texas', 1]])

OneHotEncoder 클래스는 기본적으로 희소 배열을 반환한다. sparse = False로 지정하면 밀집 배열을 얻을 수 있다.

In [15]:
one_hot_encoder = OneHotEncoder(sparse = False)
one_hot_encoder.fit_transform(feature)

array([[0., 0., 1., 1., 0.],
       [1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 1.],
       [0., 1., 0., 1., 0.],
       [0., 0., 1., 1., 0.]])

California, Delaware, Texas가 처음 세 개의 열에 원-핫 인코딩되었고, 1과 3이 나머지 두 개의 열에 원-핫 인코딩되었다.\
이 예에서처럼 정수도 문자열처럼 취급하여 변환된다.\
categories_ 속성으로 클래스를 확인할 수 있다.

In [16]:
one_hot_encoder.categories_

[array(['California', 'Delaware', 'Texas'], dtype='<U10'),
 array(['1', '3'], dtype='<U10')]

OneHotEncoder는 입력 특성 배열을 모두 범주형으로 인식하여 반환한다.\
특정 열에만 적용하려면 이전 장에서 설명한 ColumnTransformer와 함께 사용하라.

# 5.2 순서가 있는 범주형 특성 인코딩하기

In [17]:
import pandas as pd

In [18]:
dataframe = pd.DataFrame({'Score': ['Low', 'Low', 'Medium', 'Medium', 'High']})

In [25]:
# 매핑 딕셔너리
scale_mapper = {'Low': 1,
               'Medium': 2,
               'High': 3}

In [26]:
dataframe['Score'].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    3
Name: Score, dtype: int64

순서가 포함된 경우\
예시) 매우 그렇다, 그렇다, 보통이다, 그렇지 않다, 전혀 그렇지 않다.

순서가 있을 땐 수치값으로 변환하는게 일반적이다.\
어떤 수치값을 선택하는지는 클래스에 내재된 순서 정보에 기반한다는 사실을 유념해야한다.\
해결에선 high는 글자 그대로 low보다 세 배 더 크다. 경우에 따라 이런 설정이 잘 맞을 수 있지만 클래스 사이 간격이 동일하지 않는다면 문제가 된다.

In [33]:
dataframe = pd.DataFrame({'Score': ['Low',
                                    'Low',
                                    'Medium',
                                    'Medium',
                                    'High',
                                    'Barely More Than Medium']})

In [36]:
scale_mapper = {'Low': 1,
                'Medium': 2,
                'Barely More Than Medium' : 3,
                'High': 4}

In [37]:
dataframe.replace(scale_mapper)

,Score
0,1
1,1
2,2
3,2
4,4
5,3


In [39]:
# 보다 정확하게 하기 위해서 mapping할 때 좀 정밀한 수치로 만들어주기

scale_mapper = {'Low': 1,
                'Medium': 2,
                'Barely More Than Medium': 2.1,
                'High': 3}

In [40]:
dataframe.replace(scale_mapper)

,Score
0,1.0
1,1.0
2,2.0
3,2.0
4,3.0
5,2.1


In [43]:
# OrdinalEncoder을 이용하여 범주형 인코딩 해보기

from sklearn.preprocessing import OrdinalEncoder

In [48]:
features = np.array([['Low', 10],
                     ['High', 50],
                     ['Medium', 3]])
features

array([['Low', '10'],
       ['High', '50'],
       ['Medium', '3']], dtype='<U6')

In [51]:
ordinal_encoder = OrdinalEncoder()
ordinal_encoder.fit_transform(features)

array([[1., 0.],
       [0., 2.],
       [2., 1.]])

In [52]:
# 클래스 확인

ordinal_encoder.categories_

[array(['High', 'Low', 'Medium'], dtype='<U6'),
 array(['10', '3', '50'], dtype='<U6')]

# 5.3 특성 딕셔너리를 인코딩하기

In [73]:
from sklearn.feature_extraction import DictVectorizer

In [74]:
data_dict = [{'Red': 2, 'Blue': 4},
             {'Red': 4, 'Blue': 3},
             {'Red': 1, 'Yellow': 2},
             {'Red': 2, 'Yellow': 2}]

In [78]:
dicvectorizer = DictVectorizer(sparse = False)
features = dicvectorizer.fit_transform(data_dict)
features

array([[4., 2., 0.],
       [3., 4., 0.],
       [0., 1., 2.],
       [0., 2., 2.]])

DicVectorizer는 0이 아닌 값의 원소만 저장하는 희소 행렬을 반환한다.\
Dicvectorizer를 sparse = False 로 지정하면 밀집 벡터를 출력할 수 있다.

In [76]:
# 특성 이름 얻기
feature_names = dicvectorizer.get_feature_names()
feature_names

['Blue', 'Red', 'Yellow']

In [79]:
# 판다스 사용
import pandas as pd

pd.DataFrame(features, columns = feature_names)

,Blue,Red,Yellow
0,4.0,2.0,0.0
1,3.0,4.0,0.0
2,0.0,1.0,2.0
3,0.0,2.0,2.0


머신러닝 알고리즘은 행렬 형태의 데이터를 기대한다.\
DictVectorizer 는 자연어 처리 분야에서 자주 사용한다.\
DictVectorizer를 사용하면 각 문서에 등장한 단어 횟수를 특성으로 하는 특성 행렬을 만들 수 있다.

In [80]:
# 네 개의 문서에 대한 단어 카운트 딕셔너리 만들기.

doc_1_word_count = {'Red': 2, 'Blue': 4}
doc_2_word_count = {'Red': 4, 'Blue': 3}
doc_3_word_count = {'Red': 1, 'Yellow': 2}
doc_4_word_count = {'Red': 2, 'Yellow': 2}

In [81]:
# 리스트 만들기
doc_word_counts = [doc_1_word_count, doc_2_word_count, doc_3_word_count, doc_4_word_count]

In [82]:
doc_word_counts

[{'Red': 2, 'Blue': 4},
 {'Red': 4, 'Blue': 3},
 {'Red': 1, 'Yellow': 2},
 {'Red': 2, 'Yellow': 2}]

In [84]:
dicvectorizer.fit_transform(doc_word_counts)

array([[4., 2., 0.],
       [3., 4., 0.],
       [0., 1., 2.],
       [0., 2., 2.]])

이 예제는 고유한 단어가 세 개(Red, Yellow, Blue) 뿐이므로 행렬에 세 개의 특성만 있다.\
문서가 대학 도서관에 있는 어떤 책이라면 아주 큰 특성 행렬이 만들어진다(sparse 매개변수를 True로 설정해야한다.)

# 5.4 누락된 클래스 값 대체하기

In [85]:
# 일반적으로 knn 분류기를 사용한다.

import numpy as np
from sklearn.neighbors import KNeighborsClassifier

In [86]:
# 범주형 특성을 가진 행렬 만들기

X = np.array([[0, 2.10, 1.45],
              [1, 1.18, 1.33],
              [0, 1.22, 1.27],
              [1, -0.21, -1.19]])

In [87]:
# 범주형 특성에 누락된 값이 있는 특성 행렬을 만든다.

X_with_nan = np.array([[np.nan, 0.87, 1.13],
                       [np.nan, -0.67, -0.22]])

In [89]:
# knn 학습기를 훈련한다.

clf = KNeighborsClassifier(3, weights = 'distance')
trained_model = clf.fit(X[:, 1:], X[:, 0])

In [90]:
# 누락된 값의 클래스를 예측한다.

imputed_values = trained_model.predict(X_with_nan[:, 1:])

In [91]:
imputed_values

array([0., 1.])

In [92]:
# 예측된 클래스와 원본 특성을 열로 합친다.
# reshape(-1, 1): 행은 많고 열은 한개로
# hstack: 열방향으로 합치기

X_with_imputed = np.hstack((imputed_values.reshape(-1, 1), X_with_nan[:, 1:]))

In [94]:
# 두 특성 행렬을 연결한다.
# vstack: 행방향으로 합치기

np.vstack((X_with_imputed, X))

array([[ 0.  ,  0.87,  1.13],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

누락된 값을 특성에서 가장 자주 등장하는 값으로 채우기

In [95]:
from sklearn.impute import SimpleImputer

In [97]:
# 특성 행렬 합치기

X_complete = np.vstack((X_with_nan, X))

In [98]:
imputer = SimpleImputer(strategy = 'most_frequent')
imputer.fit_transform(X_complete)

array([[ 0.  ,  0.87,  1.13],
       [ 0.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

# 5.5 불균형한 클래스 다루기

데이터가 불균형할 때 더 많은 데이터를 모아라. 이것이 안되면 모델 평가 지표를 바꿔라.\
잘 동작하지 않으면 (가능하다면) 모델에 내장된 클래스 가중치 매개변수를 사용하거나 다운샘플링이나 업샘플링을 고려해보아라.

In [105]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

In [144]:
iris = load_iris()

In [145]:
features = iris.data

In [146]:
target = iris.target

In [147]:
# 처음 40개 샘플 삭제하기
features = features[40:, :]
target = target[40:]

In [152]:
# 클래스 0을 음성 클래스로 하는 이진 타깃 벡터를 만든다.
# target == 0 이면 참은 0, 거짓은 1로 바꿔라

target = np.where((target == 0), 0, 1)

In [153]:
# 불균형한 타깃 벡터 확인
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

사이킷런에 잇는 많은 알고리즘은 훈련할 때 불균형한 영향을 줄일  수 있도록 클래스에 가중치를 부여할 수 있는 매개변수를 제공한다.\
가장 인기 높은 분류 알고리즘: RF

In [154]:
# 가중치를 만든다.

weights = {0: .9, 1: 0.1}

In [155]:
# 가중치를 부여한 랜덤 포레스트 분류기를 만든다.
RandomForestClassifier(class_weight = weights)

RandomForestClassifier(class_weight={0: 0.9, 1: 0.1})

In [156]:
# balanced로 지정하여 클래스 빈도에 반비례하게 자동으로 가중치를 만들 수 있다.
# 균형잡힌 클래스 가중치로 랜덤 포레스트 모델을 훈련한다.

RandomForestClassifier(class_weight = 'balanced')

RandomForestClassifier(class_weight='balanced')

다수 클래스의 샘플을 줄이거나(다운샘플링) 소수 클래스의 샘플을 늘릴 수도 있다(업샘플링)\
- 다운샘플링: 다수 클래스에서 중복을 허용하지 않고 랜덤하게 샘플을 선택하여 소수클래스와 같은 크기의 샘플 부분집합을 만듦

In [157]:
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [158]:
# 각 클래스의 샘플 인덱스를 추출한다.

i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]

In [167]:
i_class0

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [168]:
i_class1

array([ 10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,
        23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
        36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,
        49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
        62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,
        75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
        88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
       101, 102, 103, 104, 105, 106, 107, 108, 109], dtype=int64)

In [169]:
# 각 클래스의 샘플 개수

n_class0 = len(i_class0)
n_class1 = len(i_class1)

In [170]:
# 클래스 0의 샘플만큼 클래스 1에서 중복을 허용하지 않고 랜덤하게 샘플을 뽑는다.

i_class1_downsampled = np.random.choice(i_class1, size = n_class0,
                                       replace = False)

In [171]:
# 클래스 0의 타깃 벡터와 다운샘플링된 클래스 1의 타깃 벡터를 합친다.

np.hstack((target[i_class0], target[i_class1_downsampled]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [172]:
# 클래스 0의 특성 행렬과 다운샘플링된 클래스 1의 특성 행렬을 합친다.

np.vstack((features[i_class0, :], features[i_class1_downsampled, :]))[0:5]

array([[5. , 3.5, 1.3, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4]])

다음은 소수 클래스를 업샘플링하는 방법\
업샘플링에서는 다수 클래스의 샘플만큼 소수 클래스에서 중복을 허용하여 랜덤하게 샘플을 선택한다.

In [173]:
# 클래스 1의 샘플 개수만큼 클래스 0에서 중복을 허용하여 랜덤하게 샘플을 선택한다.

i_class0_upsampled = np.random.choice(i_class0, size = n_class1, replace = True)

In [174]:
i_class0_upsampled

array([6, 6, 8, 6, 9, 1, 9, 9, 5, 4, 9, 8, 2, 0, 0, 7, 2, 5, 5, 6, 0, 8,
       3, 3, 5, 6, 7, 7, 8, 9, 9, 0, 6, 8, 7, 4, 1, 7, 5, 6, 2, 2, 8, 0,
       1, 9, 6, 0, 5, 0, 2, 7, 3, 8, 8, 3, 1, 1, 8, 6, 0, 7, 3, 8, 0, 4,
       9, 7, 5, 8, 0, 7, 1, 6, 6, 7, 9, 2, 4, 7, 4, 8, 6, 6, 9, 4, 9, 2,
       5, 7, 4, 9, 0, 3, 0, 0, 8, 7, 9, 4], dtype=int64)

In [175]:
# 클래스 0의 업샘플링된 타깃 벡터와 클래스 1의 타깃 벡터를 합친ㅇ다.

np.concatenate((target[i_class0_upsampled], target[i_class1]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [176]:
# 클래스 0의 업샘플링된 특성 행렬과 클래스 1의 특성 행렬을 합친다.

np.vstack((features[i_class0_upsampled, :], features[i_class1, :]))[0:5]

array([[5.1, 3.8, 1.6, 0.2],
       [5.1, 3.8, 1.6, 0.2],
       [5.3, 3.7, 1.5, 0.2],
       [5.1, 3.8, 1.6, 0.2],
       [5. , 3.3, 1.4, 0.2]])